# Replication of Gu, Kelly and Xiu (2020, RFS)

Current Version: Jan 2023

By [Yanlin Bao](https://www.ylbao.dev/), PhD in Business (Finance) student at Singapore Management University.

email: ylbao.2022@pbs.smu.edu.sg

This jupyter notebook is a simple version of replication code in Python of the seminal work [Gu, Kelly and Xiu (2020)](https://doi.org/10.1093/rfs/hhaa009). In this version, I mainly focus on the implementation of the models used in this paper. I hope this can be a reference for researchers, students and practitioners who want to explore the machine learning application in asset pricing. I run this version on my laptop. Therefore, I only present the result of a subsample of the original dataset, and only focus on the top 1000 firm with respect to market capitalization. I also tried for the whole firm space and the bottom 1000 firms. The results are consistent with those in the paper that the stock return of large firms are more predictable. As a result, I tried some more agressive hyperparameter setting in validation process.

Apart from the original RFS paper, for more details about the implementation, I refer the readers to the [online appendix](https://dachxiu.chicagobooth.edu/download/ML_supp.pdf) and [Q&A](https://www.dropbox.com/s/4vsc4hakwvz2j31/ML_QandA.pdf?dl=0) provided by the authors. Since the machine learning models have too many hyperparameters to choose, and the training process of some models have random initial guess for the parameters, fully replication of the results from the paper is impossible.

Besides, I believe there are some mistakes in my code, and there are still some unsolved problems. Please feel free to play the code and contact me through email.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc # garbage collection module to release memory usage in time
import time
import warnings

os.chdir('/Users/baoyanlin/Replication_Code/GKX_2020_RFS/data')

warnings.filterwarnings('ignore')

%matplotlib inline

## Data Preparation
### Firm Characteristics Data

The newest version of firm characteristics data are downloaded from Prof Dacheng Xiu's [website](https://dachxiu.chicagobooth.edu/). The data may take 4 to 6 hours to download though. Prof [Yuan Yao](https://yao-lab.github.io/) provide a dropbox [link](https://www.dropbox.com/s/zzgjdubvv23xkfp/datashare.zip?dl=0) which takes only a few minutes to download. However, the current version ends at 2021.

In [2]:
%%time
# start date and end date of the sample
#stdt, nddt = 19570101, 20161231
stdt, nddt = 20010101, 20201231

# load firm characteristics data
data_ch = pd.read_csv('GKX_20201231.csv')
data_ch = data_ch[(data_ch['DATE']>=stdt)&(data_ch['DATE']<=nddt)].reset_index(drop=True)
data_ch['DATE'] = pd.to_datetime(data_ch['DATE'],format='%Y%m%d')+pd.offsets.MonthEnd(0)
characteristics = list(set(data_ch.columns).difference({'permno','DATE','SHROUT','mve0','sic2','RET','prc'}))

data_ch.head()

CPU times: user 30.2 s, sys: 7.03 s, total: 37.2 s
Wall time: 39.7 s


,permno,DATE,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,...,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,bm,bm_ia
0,10001,2001-01-31,24355.500,0.012821,9.8750,2498,0.037079,0.001375,0.281788,8.395576,...,0.020711,1.098587e-06,0.027778,0.017710,0.972710,0.426715,4.200000e+00,49.0,0.868139,-0.104925
1,10002,2001-01-31,78332.625,0.088435,10.0000,8526,0.206346,0.042579,0.050021,8.067022,...,0.033991,6.509871e-06,0.134328,0.054790,1.368962,0.759666,4.200000e+00,60.0,0.680296,-0.152630
2,10012,2001-01-31,39836.000,0.500000,3.0000,20897,2.470629,6.104008,-1.170178,11.360419,...,0.138777,9.482216e-08,0.129412,0.075671,0.465917,7.007556,8.756593e-09,36.0,0.061049,-0.397365
3,10016,2001-01-31,379569.500,0.030726,23.0625,16964,0.449866,0.202379,0.391222,12.024414,...,0.054578,5.643552e-08,0.070769,0.040708,1.242227,8.102766,1.833562e-08,38.0,0.287808,-0.227582
4,10019,2001-01-31,28945.000,0.071429,3.7500,8270,2.249729,5.061279,0.203106,9.294773,...,0.131620,3.206363e-07,0.435897,0.120324,0.983488,16.163956,7.497863e-09,38.0,0.552262,0.036872


### Pick out Top 1000 and Bottom 1000 Firms

Next, let's pick out the top 1000 and bottom 1000 firms with respect to market capitalization to see the differnce of predictability between big firms and small firms.

In [3]:
data_ch_top = data_ch.sort_values('mvel1',ascending=False).groupby('DATE').head(1000).reset_index(drop=True)
data_ch_bot = data_ch.sort_values('mvel1',ascending=False).groupby('DATE').tail(1000).reset_index(drop=True)

### Missing Characteristics
According to the paper, the __missing data__ are replaced by the _cross-sectional median_.

In [4]:
# missing data before filling
data_ch.isnull().sum()

permno            0
DATE              0
mvel1           276
RET               0
prc            7830
              ...  
std_turn        410
zerotrade       342
sic2          22532
bm           375825
bm_ia        375825
Length: 101, dtype: int64

In [5]:
%%time
# fill na with cross-sectional median
for ch in characteristics:
     data_ch[ch] = data_ch.groupby('DATE')[ch].transform(lambda x: x.fillna(x.median()))

CPU times: user 12.7 s, sys: 11.6 s, total: 24.3 s
Wall time: 24.3 s


In [6]:
# missing data after filling
data_ch.isnull().sum()

permno            0
DATE              0
mvel1             0
RET               0
prc            7830
              ...  
std_turn          0
zerotrade         0
sic2          22532
bm           104764
bm_ia        104764
Length: 101, dtype: int64

Since there are some characeristics that are all missing at some time point, we still encounter missing data after the filling process. Then, let's try to fill the remaining na with time-series median. Unfortunately, after filling na with time-series median, na still exists. Since there is no further instruction of how to deal with remaining na in the data, after consulting some replication code online, I __fill the remaining na with 0__.

In [7]:
for ch in characteristics:
     data_ch[ch] = data_ch[ch].fillna(0)
    
data_ch.columns[data_ch.isnull().sum()!=0]

Index(['prc', 'mve0', 'sic2'], dtype='object')

Now, we do not have missing characteristics in our dataset. Then, do the same process to top and bottom 1000 firms data.

In [8]:
def fill_na(data_ch, characteristics):
    for ch in characteristics:
         data_ch[ch] = data_ch.groupby('DATE')[ch].transform(lambda x: x.fillna(x.median()))
    for ch in characteristics:
         data_ch[ch] = data_ch[ch].fillna(0)
    return data_ch

In [9]:
data_ch_top = fill_na(data_ch_top, characteristics)
data_ch_bot = fill_na(data_ch_bot, characteristics)

### Transform SIC Code into Dummies

In [10]:
# get dummies for SIC code
def get_sic_dummies(data_ch):
    sic_dummies = pd.get_dummies(data_ch['sic2'].fillna(999).astype(int),prefix='sic').drop('sic_999',axis=1)
    data_ch_d = pd.concat([data_ch,sic_dummies],axis=1)
    data_ch_d.drop(['prc','SHROUT','mve0','sic2'],inplace=True,axis=1)
    return data_ch_d

In [11]:
data_ch_d = get_sic_dummies(data_ch)
data_ch_top_d = get_sic_dummies(data_ch_top)
data_ch_bot_d = get_sic_dummies(data_ch_bot)

### Macroeconomic Predictors Data

The eight macroeconomic predictors follows the definitions by Welch and Goyal (2008, RFS). The data are available on Prof Goyal's [website](https://sites.google.com/view/agoyal145).

In [12]:
# load macroeconomic predictors data
data_ma = pd.read_csv('PredictorData2021.csv')
data_ma = data_ma[(data_ma['yyyymm']>=stdt//100)&(data_ma['yyyymm']<=nddt//100)].reset_index(drop=True)

# construct predictor
ma_predictors = ['dp_sp','ep_sp','bm_sp','ntis','tbl','tms','dfy','svar']
data_ma['Index'] = data_ma['Index'].str.replace(',','').astype('float64')
data_ma['dp_sp'] = data_ma['D12']/data_ma['Index']
data_ma['ep_sp'] = data_ma['E12']/data_ma['Index']
data_ma.rename({'b/m':'bm_sp'},axis=1,inplace=True)
data_ma['tms'] = data_ma['lty']-data_ma['tbl']
data_ma['dfy'] = data_ma['BAA']-data_ma['AAA']
data_ma = data_ma[['yyyymm']+ma_predictors]
data_ma['yyyymm'] = pd.to_datetime(data_ma['yyyymm'],format='%Y%m')+pd.offsets.MonthEnd(0)
data_ma.head()

,yyyymm,dp_sp,ep_sp,bm_sp,ntis,tbl,tms,dfy,svar
0,2001-01-31,0.011839,0.035490,0.150450,-0.003193,0.0515,0.0047,0.0078,0.004941
1,2001-02-28,0.012962,0.037873,0.156070,-0.006856,0.0488,0.0061,0.0077,0.002528
2,2001-03-31,0.013766,0.039161,0.133114,-0.005213,0.0442,0.0117,0.0086,0.007140
3,2001-04-30,0.012707,0.034060,0.122497,-0.002543,0.0387,0.0206,0.0087,0.007426
4,2001-05-31,0.012567,0.031592,0.120510,-0.000248,0.0362,0.0232,0.0078,0.002536


### Construct the Dataset including all the Features

Besides adding the interaction terms, this function also transform the data into (-1, 1).

In [13]:
from sklearn.preprocessing import MinMaxScaler

def interactions(data_ch, data_ma, characteristics, ma_predictors, minmax=True):
    # construct interactions between firm characteristics and macroeconomic predictors
    data = data_ch.copy()
    data_ma_long = pd.merge(data[['DATE']],data_ma,left_on='DATE',right_on='yyyymm',how='left')
    data = data.reset_index(drop=True)
    data_ma_long = data_ma_long.reset_index(drop=True)
    for fc in characteristics:
        for mp in ma_predictors:
            data[fc+'*'+mp] = data[fc]*data_ma_long[mp]

    features = list(set(data.columns).difference({'permno','DATE','RET'})) # a list storing all 920 features used
    if minmax:
        X = MinMaxScaler((-1,1)).fit_transform(data[features])
        X = pd.DataFrame(X, columns=features)
    else:
        X = data[features]
    y = data['RET']
    print(f"The shape of the data is: {data.shape}")
    return X, y

### Split the Sample into Training Set, Validation Set and Testing Set

According to the paper, the authors use first 18 years (1957-1974) for training, last 30 years (1987-2016) for out-of-sample testing, and the 12 years in the middle (1975-1986) for tuning hyperparameters.

In [14]:
#stdt_vld = np.datetime64('1975-01-31')
#stdt_tst = np.datetime64('1987-01-31')
stdt_vld = np.datetime64('2009-01-31')
stdt_tst = np.datetime64('2015-01-31')

def trn_vld_tst(data):

    # training setstdt_vld = np.datetime64('2001-01-31')
    X_trn, y_trn = interactions(data[data['DATE']<stdt_vld],data_ma[data_ma['yyyymm']<stdt_vld],characteristics,ma_predictors)

    # validation set
    X_vld, y_vld = interactions(data[(data['DATE']<stdt_tst)&(data['DATE']>=stdt_vld)],data_ma[(data_ma['yyyymm']<stdt_tst)&(data_ma['yyyymm']>=stdt_vld)],characteristics,ma_predictors)

    # testing set
    X_tst, y_tst = interactions(data[data['DATE']>=stdt_tst],data_ma[data_ma['yyyymm']>=stdt_tst],characteristics,ma_predictors)
    return X_trn, X_vld, X_tst, y_trn, y_vld, y_tst

In [15]:
%%time

X_trn, X_vld, X_tst, y_trn, y_vld, y_tst = trn_vld_tst(data_ch_top_d)

The shape of the data is: (96000, 915)
The shape of the data is: (72000, 915)
The shape of the data is: (72000, 915)
CPU times: user 3.14 s, sys: 2.06 s, total: 5.2 s
Wall time: 5.2 s


The differnce in the number of features results from the number of dummies from SIC code in the sample period.

In [16]:
X_trn.head()

,cfp*dfy,bm*tms,sic_52,currat*ep_sp,currat*bm_sp,lgr,dolvol*dfy,sic_61,zerotrade*ep_sp,chinv*dfy,...,retvol*dp_sp,pctacc*ntis,pchsale_pchinvt*dp_sp,pricedelay*svar,pchsale_pchinvt*ep_sp,turn*tbl,sic_34,herf*dfy,roic*ntis,pchcapx_ia*tbl
0,0.116731,-0.991857,-1.0,-0.963915,-0.972437,-0.907963,-0.549709,-1.0,-1.0,-0.025066,...,-0.889148,-0.059615,0.473791,-0.040381,0.437677,-0.906161,-1.0,-0.989795,-0.410517,0.884035
1,0.144025,-0.991877,-1.0,-0.968719,-0.976182,-0.953632,-0.552978,1.0,-1.0,-0.059365,...,-0.960461,-0.052912,0.575258,-0.040451,0.578087,-0.971478,-1.0,-0.895286,-0.401403,0.857572
2,0.125601,-0.991823,1.0,-0.983676,-0.987842,-0.873205,-0.549709,-1.0,-1.0,0.010791,...,-0.957749,-0.060920,0.499423,-0.040361,0.473146,-0.962165,-1.0,-0.701821,-0.411373,0.889706
3,0.155076,-0.991608,-1.0,-0.981250,-0.985950,-0.788581,-0.560747,-1.0,-1.0,-0.040978,...,-0.932604,-0.033102,0.478819,-0.039828,0.444635,-0.963900,-1.0,-0.987912,-0.401405,0.888456
4,0.178162,-0.991670,-1.0,-0.987563,-0.990872,-0.887720,-0.549709,-1.0,-1.0,-0.047678,...,-0.941044,-0.059871,0.541947,-0.040542,0.531991,-0.959844,-1.0,-0.929853,-0.407750,0.875599


In [17]:
gc.collect()

0

In [18]:
del([data_ch,data_ch_top,data_ch_bot,data_ch_d,data_ch_top_d,data_ch_bot_d])

## Model Fitting

In total, 8 models, including ordinary least squares (__OLS__), partial least squares (__PLS__), principal component regression (__PCR__), elastic net (__ENet__), generalized linear model with group lasso (__GLM__), random forest (__RF__), gradient boosting regression trees (__GBRT__) and neural networks (__NN__), are implemented in this paper. The objective loss functions are mean squared errors and Huber robust objective function that substitute squared loss with absolute loss for outliers. The training process is done in the training set, and the hyperparameters are tuned in the validation set.

Due to the limitation of computational power, I mainly use the default or preselected hyperparameters when training the model. However, as the target of this replication is to get familiar with the whole process of implementing machine learning methods for empirical asset pricing research, I will conduct tuning process for some simple models (e.g., tuning the parameter $\xi$ in Huber loss function when using preselected features including size, bm and momentum). Besides, as scikit-learn does not provide the flexibility to customize loss function in the training process, I will also write a customized code to incorporate Huber loss with elastic net.

### Customized Loss Function, Scoring Functions, Validation Funtion and Evaluation Function

In [19]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import ParameterGrid

# Loss Function
# Huber objective function
def huber(actual, predicted, xi):
    actual, predicted = np.array(actual).flatten(), np.array(predicted).flatten()
    resid = actual - predicted
    huber_loss = np.where(np.abs(resid)<=xi, diff**2, 2*xi*np.abs(resid)-xi**2)
    return np.mean(huber_loss)

# Scoring Function
# out-of-sample R squared
def R_oos(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted).flatten()
    predicted = np.where(predicted<0,0,predicted)
    return 1 - (np.dot((actual-predicted),(actual-predicted)))/(np.dot(actual,actual))

# Validation Function
def val_fun(model, params: dict, X_trn, y_trn, X_vld, y_vld, illustration=True, sleep=0, is_NN=False):
    best_ros = None
    lst_params = list(ParameterGrid(params))
    for param in lst_params:
        if best_ros == None:
            if is_NN:
                mod = model().set_params(**param).fit(X_trn, y_trn, X_vld, y_vld)
            else:
                mod = model().set_params(**param).fit(X_trn, y_trn)
            best_mod = mod
            y_pred = mod.predict(X_vld)
            best_ros = R_oos(y_vld, y_pred)
            best_param = param
            if illustration:
                print(f'Model with params: {param} finished.')
                print(f'with out-of-sample R squared on validation set: {best_ros*100:.5f}%')
                print('*'*60)
        else:
            time.sleep(sleep)
            if is_NN:
                mod = model().set_params(**param).fit(X_trn, y_trn, X_vld, y_vld)
            else:
                mod = model().set_params(**param).fit(X_trn, y_trn)
            y_pred = mod.predict(X_vld)
            ros = R_oos(y_vld, y_pred)
            if illustration:
                print(f'Model with params: {param} finished.')
                print(f'with out-of-sample R squared on validation set: {ros*100:.5f}%')
                print('*'*60)
            if ros > best_ros:
                best_ros = ros
                best_mod = mod
                best_param = param
    if illustration:
        print('\n'+'#'*60)
        print('Tuning process finished!!!')
        print(f'The best setting is: {best_param}')
        print(f'with R2oos {best_ros*100:.2f}% on validation set.')
        print('#'*60)
    return best_mod
    

# Pairwise Comparison
# Diebold-Mariano test statistics

# Evaluation Output
def evaluate(actual, predicted, insample=False):
    if insample:
        print('*'*15+'In-Sample Metrics'+'*'*15)
        print(f'The in-sample R2 is {r2_score(actual,predicted)*100:.2f}%')
        print(f'The in-sample MSE is {mean_squared_error(actual,predicted):.3f}')
    else:
        print('*'*15+'Out-of-Sample Metrics'+'*'*15)
        print(f'The out-of-sample R2 is {R_oos(actual,predicted)*100:.2f}%')
        print(f'The out-of-sample MSE is {mean_squared_error(actual,predicted):.3f}')

### OLS
#### MSE as objective function

In [20]:
%%time
time.sleep(10)
from sklearn.linear_model import LinearRegression

# OLS with all features
OLS = LinearRegression().fit(X_trn,y_trn)
evaluate(y_trn, OLS.predict(X_trn), insample=True)
evaluate(y_tst, OLS.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 19.40%
The in-sample MSE is 0.009
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -88.69%
The out-of-sample MSE is 0.053
CPU times: user 18.4 s, sys: 2.97 s, total: 21.3 s
Wall time: 14.5 s


In [21]:
%%time
from sklearn.linear_model import LinearRegression

# OLS with preselected size, bm, and momentum covariates
features_3 = ['mvel1','bm','mom1m','mom6m','mom12m','mom36m']
OLS_3 = LinearRegression().fit(X_trn[features_3],y_trn)
evaluate(y_trn, OLS_3.predict(X_trn[features_3]), insample=True)
evaluate(y_tst, OLS_3.predict(X_tst[features_3]))

***************In-Sample Metrics***************
The in-sample R2 is 0.45%
The in-sample MSE is 0.011
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.45%
The out-of-sample MSE is 0.009
CPU times: user 58.4 ms, sys: 11.2 ms, total: 69.6 ms
Wall time: 15.4 ms


In [22]:
gc.collect()

0

#### Huber objective fuction

The authors choose $\xi$ as 99.9% percentile of the pricing error.

In [23]:
%%time
time.sleep(10)
from sklearn.linear_model import HuberRegressor

# OLS by Huber robust objective function with all features
epsilon = np.max(((y_trn-OLS.predict(X_trn)).quantile(.999),1))
OLS_H = HuberRegressor(epsilon=epsilon).fit(X_trn,y_trn)
evaluate(y_trn, OLS_H.predict(X_trn), insample=True)
evaluate(y_tst, OLS_H.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 13.48%
The in-sample MSE is 0.010
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -3.05%
The out-of-sample MSE is 0.010
CPU times: user 3min 5s, sys: 38.9 s, total: 3min 44s
Wall time: 48.4 s


In [24]:
%%time
from sklearn.linear_model import HuberRegressor

# OLS by Huber robust objective function
# with preselected size, bm, and momentum covariates
epsilon = np.max(((y_trn-OLS_3.predict(X_trn[features_3])).quantile(.999),1))
features_3 = ['mvel1','bm','mom1m','mom6m','mom12m','mom36m']
OLS_H_3 = HuberRegressor(epsilon=epsilon).fit(X_trn[features_3],y_trn)

evaluate(y_trn, OLS_H_3.predict(X_trn[features_3]), insample=True)
evaluate(y_tst, OLS_H_3.predict(X_tst[features_3]))

***************In-Sample Metrics***************
The in-sample R2 is 0.23%
The in-sample MSE is 0.011
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.61%
The out-of-sample MSE is 0.009
CPU times: user 1.22 s, sys: 237 ms, total: 1.46 s
Wall time: 267 ms


In [25]:
gc.collect()

57

### PLS

In [26]:
%%time
from sklearn.cross_decomposition import PLSRegression

params = {'n_components': [1, 5, 10, 50]}
PLS = val_fun(PLSRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'n_components': 1} finished.
with out-of-sample R squared on validation set: 0.00040%
************************************************************
Model with params: {'n_components': 5} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'n_components': 10} finished.
with out-of-sample R squared on validation set: -0.00385%
************************************************************
Model with params: {'n_components': 50} finished.
with out-of-sample R squared on validation set: -1927.66308%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'n_components': 1}
with R2oos 0.00% on validation set.
############################################################
CPU times: user 2min 27s, sys: 22.5 s, total: 2min 49s
Wall time: 1min 14s


As the result suggests, let's use only 1 component in PLS.

In [27]:
%%time
pls_pred_is = PLS.predict(X_trn)
pls_pred_os = PLS.predict(X_tst)
evaluate(y_trn, pls_pred_is, insample=True) 
evaluate(y_tst, pls_pred_os)

***************In-Sample Metrics***************
The in-sample R2 is 5.70%
The in-sample MSE is 0.011
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.02%
The out-of-sample MSE is 0.009
CPU times: user 1.57 s, sys: 668 ms, total: 2.24 s
Wall time: 492 ms


In [28]:
gc.collect()

0

### PCR

In [29]:
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

class PCRegressor:
    
    def __init__(self,n_PCs=1,loss='mse'):
        self.n_PCs = n_PCs
        if loss not in ['huber','mse']:
            raise AttributeError(
            f"The loss should be either 'huber' or 'mse', but {loss} is given"
            )
        else:
            self.loss = loss
        
    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
        
    def fit(self,X,y):
        X = np.array(X)
        N,K = X.shape
        y = np.array(y_trn).reshape((N,1))
        self.mu = np.mean(X,axis=0).reshape((1,K))
        self.sigma = np.std(X,axis=0).reshape((1,K))
        self.sigma = np.where(self.sigma==0,1,self.sigma)
        X = (X-self.mu)/self.sigma
        pca = PCA()
        X = pca.fit_transform(X)[:,:self.n_PCs]
        self.pc_coef = pca.components_.T[:,:self.n_PCs]
        if self.loss == 'mse':
            self.model = LinearRegression().fit(X,y)
        else:
            self.model = HuberRegressor().fit(X,y)
        return self
    
    def predict(self,X):
        X = np.array(X)
        X = (X-self.mu)/self.sigma
        X = X @ self.pc_coef
        return self.model.predict(X)

In [30]:
%%time
# principal component regression
params = {'n_PCs':[1,3,5,7,10,50],'loss':['mse','huber']}
PCR = val_fun(PCRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,sleep=3)

Model with params: {'loss': 'mse', 'n_PCs': 1} finished.
with out-of-sample R squared on validation set: 0.06330%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 3} finished.
with out-of-sample R squared on validation set: 0.03821%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 5} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 7} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 10} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'loss': 'mse', 'n_PCs': 50} finished.
with out-of-sample R squared on validation set: 0.62278%
*********

As the result suggests, let's use 3 principal components in PCR.

In [31]:
gc.collect()

174

In [32]:
%%time
evaluate(y_trn, PCR.predict(X_trn), insample=True) 
evaluate(y_tst, PCR.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 6.88%
The in-sample MSE is 0.011
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -0.01%
The out-of-sample MSE is 0.010
CPU times: user 2.25 s, sys: 1.57 s, total: 3.82 s
Wall time: 2.02 s


In [33]:
gc.collect()

0

### Elastic Net

Since scikit-learn does not provide the flexibility of choosing objective function, I code this part refering to the accelerated proximal algorithm (APG) documented in the online appendix. Besides, I see lasso and ridge as a special case of elastic net.

I use the setting from the paper, where the regularized objective function is:
$$L(\theta;\cdot) = L(\theta) + \phi(\theta;\cdot)$$
where $\theta$ is the vector of parameters, $L(\theta)$ is the loss funciton (MSE or Huber loss in our case), and $\phi(\theta;\cdot)$ is the penalty.

Specifically, for our elastic net:
$$\phi(\theta;\lambda,\rho)=\lambda(1-\rho)\sum_{j=1}^{P}|\theta_{j}|+\frac{1}{2}\lambda\rho\sum_{j=1}^{P}\theta_{j}^{2}$$

As for the details of APG, I refer the readers to the online appendix of the paper, which can be found on Prof Xiu's [website](https://dachxiu.chicagobooth.edu/). The parameter $\gamma$ is the learning rate.

In [34]:
# mse
def mse(actual, predicted):
    actual, predicted = np.array(actual).flatten(), np.array(predicted).flatten()
    resid = actual - predicted
    return np.mean(resid**2)

# huber objective function
def huber(actual, predicted, xi):
    actual, predicted = np.array(actual).flatten(), np.array(predicted).flatten()
    resid = actual - predicted
    huber_loss = np.where(np.abs(resid)<=xi, diff**2, 2*xi*np.abs(resid)-xi**2)
    return np.mean(huber_loss)

# gradient of mse
def grad_mse(X, y, theta):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    theta = np.array(theta).reshape((K,1))
    return (X.T @ (y - X@theta))/N

# gradient of huber loss
def grad_huber(X, y, theta, xi):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    theta = np.array(theta).reshape((K,1))
    resid = y - X@theta
    ind_m = np.where(np.abs(resid)<=xi)
    ind_u = np.where(resid>xi)
    ind_l = np.where(resid< -xi)
    try:
        grad_m = X[ind_m].T @ (y[ind_m] - X[ind_m]@theta)
    except:
        grad_m = np.zeros((K,1))
    try:
        grad_u = 2*xi* X[ind_u].T@np.ones((len(ind_u[0]),1))
    except:
        grad_u = np.zeros((K,1))
    try:
        grad_l = -2*xi* X[ind_l].T@np.ones((len(ind_l[0]),1))
    except:
        grad_l = np.zeros((K,1))
    return (grad_m+grad_u+grad_l)/N

# proximal operator
def prox(theta,lmd,rho,gamma):
    return (1/(1+lmd*gamma*rho))*softhred(theta,(1-rho)*gamma*lmd)

# soft-thresholding operator
def softhred(x,mu):
    x = np.where(np.abs(x)<=mu, 0, x)
    x = np.where((np.abs(x)>mu) & (x>0), x-mu, x)
    x = np.where((np.abs(x)>mu) & (x<0), x+mu, x)
    return x

# Elastic Net
class ENet:
    
    def __init__(
        self, lmd=1, rho=0.5, L=1, verbose=False,
        xi=1.35, max_iter=3000, tol=1e-4, loss='huber', random_state=None
    ):
        self.lmd = lmd
        self.rho = rho
        self.L = L
        self.verbose = verbose
        self.xi = xi
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        if loss not in ['huber','mse']:
            raise AttributeError(
            f"The loss should be either 'huber' or 'mse', but {loss} is given"
            )
        else:
            self.loss = loss
            
    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
            
    def fit(self, X, y):
        K = X.shape[1]
        X = np.array(X)
        N = len(y)
        y = np.array(y).reshape((N,1))
        gamma = 1/self.L
        # initialize theta
        if self.random_state != None:
            np.random.seed(self.random_state)
            theta = np.random.uniform(size=(K,1))
        else:
            theta = np.zeros((K,1))
        
        for m in np.arange(self.max_iter):
            theta_old = theta
            
            if self.loss == 'mse':
                theta_bar = theta - gamma*grad_mse(X,y,theta)
            else:
                theta_bar = theta - gamma*grad_huber(X,y,theta,self.xi)
                
            theta_til = prox(theta_bar,self.lmd,self.rho,gamma)
            theta = theta_til + m/(m+3)*(theta_til-theta)
            gamma = gamma
            
            if self.verbose:
                print(f'{m+1} iters finished.')
                print(f'theta = {theta.T}')
                print(f'{np.sum((theta-theta_old)**2)}')
            
            if np.sum((theta-theta_old)**2)<np.sum(theta_old**2*self.tol) or np.sum(np.abs(theta-theta_old))==0:
                break
        self.theta = theta_old
        return self
    
    def predict(self, X):
        X = np.array(X)
        return X@self.theta

In [35]:
%%time
from numpy.linalg import svd

# compute L
L = np.max(svd(X_trn,compute_uv=False))**2

params = {
    'lmd':[1e-4,.1],
    'L':[L],
    'loss':['mse']
}

EN_my_mse = val_fun(ENet,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'L': 43030456.772059955, 'lmd': 0.0001, 'loss': 'mse'} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'L': 43030456.772059955, 'lmd': 0.1, 'loss': 'mse'} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'L': 43030456.772059955, 'lmd': 0.0001, 'loss': 'mse'}
with R2oos 0.00% on validation set.
############################################################
CPU times: user 2min 37s, sys: 31 s, total: 3min 8s
Wall time: 27.2 s


In [36]:
%%time
evaluate(y_trn, EN_my_mse.predict(X_trn), insample=True) 
evaluate(y_tst, EN_my_mse.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is -0.03%
The in-sample MSE is 0.012
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.00%
The out-of-sample MSE is 0.009
CPU times: user 1.41 s, sys: 494 ms, total: 1.91 s
Wall time: 366 ms


In [37]:
gc.collect()

0

In [38]:
%%time
xi = np.max((1,(y_trn-EN_my_mse.predict(X_trn).flatten()).quantile(.999)))
params = {
    'lmd':list(np.linspace(1e-4,1e-3,10)),
    'L':[2*xi],
    #'L':[2],
    'loss':['huber']
}
EN_my_hub = val_fun(ENet,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'L': 2.0, 'lmd': 0.0001, 'loss': 'huber'} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'L': 2.0, 'lmd': 0.00019999999999999998, 'loss': 'huber'} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'L': 2.0, 'lmd': 0.0003, 'loss': 'huber'} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'L': 2.0, 'lmd': 0.00039999999999999996, 'loss': 'huber'} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'L': 2.0, 'lmd': 0.0005, 'loss': 'huber'} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params:

In [39]:
gc.collect()

0

However, the choice of learning rate $\gamma$ in APG is quite cruicial. According to [Parikh and Boyd (2013)](https://web.stanford.edu/~boyd/papers/pdf/prox_algs.pdf), the algorithm converges when $\gamma \in (0, 1/L]$, where $L$ is the _Lipschitz_ constant of the gradient of the objective funtion. As for mse, L is the square of the largest singular value of the dataset. Though my code for APG seems to work, it returns weights with same value. The readers can check this by setting the parameter _verbose_ as _False_. Therefore, I first use the elastic net provided by sklearn for the elastic net with MSE as objective function. Then, I write the fitting process as an unconstraint optimization using the optimization package from scipy. However, as I use Nelder-Mead as solver, it takes much time to get the results.

__Elastic Net from sklearn__

In [40]:
%%time
time.sleep(10)
from sklearn.linear_model import ElasticNet

params = {'alpha':[1e-4,.1]}
EN_sk = val_fun(ElasticNet,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'alpha': 0.0001} finished.
with out-of-sample R squared on validation set: -79.93287%
************************************************************
Model with params: {'alpha': 0.1} finished.
with out-of-sample R squared on validation set: 0.62828%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'alpha': 0.1}
with R2oos 0.63% on validation set.
############################################################
CPU times: user 2min 33s, sys: 4.74 s, total: 2min 38s
Wall time: 33.8 s


In [41]:
%%time
evaluate(y_trn, EN_sk.predict(X_trn), insample=True) 
evaluate(y_tst, EN_sk.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 0.00%
The in-sample MSE is 0.012
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.35%
The out-of-sample MSE is 0.009
CPU times: user 245 ms, sys: 17.6 ms, total: 263 ms
Wall time: 70.6 ms


__Elastic Net with Huber Loss by Scipy__

Next, I formulate this problem in the framework of scipy's optimization package. Though BFGS solver uses gradient and will be faster than Nelder-Mead, I encounter a runtime error _'Desired error not necessarily achieved due to precision loss'_.

In [42]:
from scipy.optimize import minimize
from functools import partial

# penalized mse
def mse_pnl(theta, X, y, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    resid = y - X@theta.reshape((K,1))
    return np.mean(resid**2) + lmd*(1-rho)*np.sum(np.abs(theta)) + 0.5*lmd*rho*np.sum(theta**2)

# penalized huber objective function
def huber_pnl(theta, X, y, xi, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    resid = y - X@theta.reshape((K,1))
    huber_loss = np.where(np.abs(resid)<=xi, resid**2, 2*xi*np.abs(resid)-xi**2)
    return np.mean(huber_loss) + lmd*(1-rho)*np.sum(np.abs(theta)) + 0.5*lmd*rho*np.sum(theta**2)

# gradient of mse
def grad_mse(theta, X, y, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    theta = np.array(theta).reshape((K,1))
    grad = (X.T @ (y - X@theta))/N + lmd*(1-rho)*np.where(theta>0,1,-1) + lmd*rho*theta
    return grad.flatten()

# gradient of huber loss
def grad_huber(theta, X, y, xi, lmd, rho):
    K = X.shape[1]
    X = np.array(X)
    N = len(y)
    y = np.array(y).reshape((N,1))
    theta = np.array(theta).reshape((K,1))
    resid = y - X@theta
    ind_m = np.where(np.abs(resid)<=xi)
    ind_u = np.where(resid>xi)
    ind_l = np.where(resid< -xi)
    try:
        grad_m = X[ind_m].T @ (y[ind_m] - X[ind_m]@theta)
    except:
        grad_m = np.zeros((K,1))
    try:
        grad_u = 2*xi* X[ind_u].T@np.ones((len(ind_u[0]),1))
    except:
        grad_u = np.zeros((K,1))
    try:
        grad_l = -2*xi* X[ind_l].T@np.ones((len(ind_l[0]),1))
    except:
        grad_l = np.zeros((K,1))
    grad = (grad_m+grad_u+grad_l)/N + lmd*(1-rho)*np.where(theta>0,1,-1) + lmd*rho*theta
    return grad.flatten()

# Elastic Net
class ENet:
    
    def __init__(
        self, lmd, rho, xi=1.35, loss='huber', random_state=None, fit_intercept=True
    ):
        self.lmd = lmd
        self.rho = rho
        self.xi = xi
        self.random_state = random_state
        self.fit_intercept = fit_intercept
        if loss not in ['huber','mse']:
            raise AttributeError(
            f"The loss should be either 'huber' or 'mse', but {loss} is given"
            )
        else:
            self.loss = loss
            
    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
            
    def fit(self, X, y):
        K = X.shape[1]
        X = np.array(X)
        N = len(y)
        y = np.array(y).reshape((N,1))
        if self.fit_intercept:
            K += 1
            X = np.concatenate((np.ones((N,1)),X),axis=1)
        # initialize theta
        if self.random_state != None:
            np.random.seed(self.random_state)
            theta = np.random.uniform(K)
        else:
            theta = np.zeros(K)
        
        if self.loss == 'huber':
            res = minimize(
                partial(huber_pnl, X=X, y=y, xi=self.xi, lmd=self.lmd, rho=self.rho), theta,
                method='nelder-mead',
                #method='BFGS',
                #jac = partial(grad_huber, X=X, y=y, xi=self.xi, lmd=self.lmd, rho=self.rho),
                options = {'disp': True}
            )
        else:
            res = minimize(
                partial(mse_pnl, X=X, y=y, lmd=self.lmd, rho=self.rho), theta, 
                method='nelder-mead',
                #method='BFGS',
                #jac = partial(grad_mse, X=X, y=y, lmd=self.lmd, rho=self.rho),
                options = {'disp': True}
            )
        
        self.theta = res.x.reshape((K,1))
        return self
    
    def predict(self, X):
        X = np.array(X)
        N = X.shape[0]
        if self.fit_intercept:
            X = np.concatenate((np.ones((N,1)),X),axis=1)
        return X@self.theta

As the Nelder-Mead solver is really slow, I randomly select 50 features to try whther my code works. I can foresee that my code will be much slower than sklearn's elastic net. Besides, as the solvers are different and the stop criteria are different, the results might be different.

In [43]:
%%time
time.sleep(10)
import random

# randomly select 50 
random.seed(12308)
features_rdm = random.sample(list(X_trn.columns),50)

# fit elastic net with mse using my code
EN_my_mse_rdm = ENet(lmd=.01,rho=.5,loss='mse').fit(X_trn[features_rdm],y_trn)
# the fitted coefficients
EN_my_mse_rdm.theta.flatten()

Optimization terminated successfully.
         Current function value: 0.011510
         Iterations: 990
         Function evaluations: 1427
CPU times: user 24.5 s, sys: 614 ms, total: 25.2 s
Wall time: 13.2 s


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [44]:
# the 50 features selected
np.array(features_rdm)

array(['idiovol*dfy', 'convind*bm_sp', 'secured*ntis', 'invest',
       'stdacc*ep_sp', 'tang*dfy', 'orgcap*ep_sp', 'salecash*tbl',
       'ps*svar', 'mvel1*tbl', 'sic_53', 'sp', 'bm', 'nincr*ntis',
       'grcapx*ntis', 'mom12m*dp_sp', 'pchgm_pchsale', 'sic_29', 'sic_37',
       'chtx*tms', 'pchcapx_ia*svar', 'agr*ntis', 'baspread*svar',
       'pricedelay', 'divo*ep_sp', 'salerec*bm_sp', 'chempia*bm_sp',
       'divi*svar', 'sp*ep_sp', 'sic_48', 'rd_mve*tbl', 'quick*ep_sp',
       'chinv*tbl', 'rsup*dp_sp', 'salerec*tbl', 'mvel1*ep_sp',
       'pricedelay*dfy', 'cash*bm_sp', 'agr*tms', 'mve_ia*tms',
       'gma*bm_sp', 'sic_17', 'absacc', 'dy*tms', 'agr*bm_sp', 'cinvest',
       'depr*ep_sp', 'rd_sale*dp_sp', 'tb*ep_sp', 'hire*tbl'],
      dtype='<U15')

In [45]:
%%time
evaluate(y_trn, EN_my_mse_rdm.predict(X_trn[features_rdm]), insample=True) 
evaluate(y_tst, EN_my_mse_rdm.predict(X_tst[features_rdm]))

***************In-Sample Metrics***************
The in-sample R2 is -0.03%
The in-sample MSE is 0.012
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.00%
The out-of-sample MSE is 0.009
CPU times: user 66.7 ms, sys: 14.6 ms, total: 81.3 ms
Wall time: 23.1 ms


In [46]:
%%time
# fit elastic net with mse by sklearn
EN_sk_rdm = ElasticNet(alpha=.01).fit(X_trn[features_rdm],y_trn)
# fitted coefficients
np.array([EN_sk_rdm.intercept_]+list(EN_sk_rdm.coef_))

CPU times: user 96 ms, sys: 11.8 ms, total: 108 ms
Wall time: 28.6 ms


array([ 0.0018227, -0.       , -0.       ,  0.       , -0.       ,
       -0.       , -0.       ,  0.       ,  0.       , -0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       , -0.       ,
        0.       ,  0.       , -0.       , -0.       ,  0.       ,
        0.       ,  0.       , -0.       , -0.       ,  0.       ,
       -0.       ,  0.       , -0.       ,  0.       ,  0.       ,
        0.       ,  0.       , -0.       , -0.       ,  0.       ,
       -0.       , -0.       , -0.       ,  0.       , -0.       ,
        0.       , -0.       , -0.       , -0.       ,  0.       ,
       -0.       ])

In [47]:
%%time
evaluate(y_trn, EN_sk_rdm.predict(X_trn[features_rdm]), insample=True) 
evaluate(y_tst, EN_sk_rdm.predict(X_tst[features_rdm]))

***************In-Sample Metrics***************
The in-sample R2 is 0.00%
The in-sample MSE is 0.012
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.35%
The out-of-sample MSE is 0.009
CPU times: user 41.6 ms, sys: 8.28 ms, total: 49.9 ms
Wall time: 11.3 ms


Though I have foreseen that the results might be different between my code and sklearn, the difference is beyond my expectation. Anyway, let's try my code with huber loss. We should expect some improvement compared with mse.

In [48]:
%%time
time.sleep(10)

# fit elastic net with huber loss using my code
EN_my_hub_rdm = ENet(lmd=.01,rho=.5,loss='huber').fit(X_trn[features_rdm],y_trn)

Optimization terminated successfully.
         Current function value: 0.011509
         Iterations: 972
         Function evaluations: 1404
CPU times: user 26.3 s, sys: 768 ms, total: 27.1 s
Wall time: 13.4 s


In [49]:
%%time
evaluate(y_trn, EN_my_hub_rdm.predict(X_trn[features_rdm]), insample=True) 
evaluate(y_tst, EN_my_hub_rdm.predict(X_tst[features_rdm]))

***************In-Sample Metrics***************
The in-sample R2 is -0.03%
The in-sample MSE is 0.012
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.00%
The out-of-sample MSE is 0.009
CPU times: user 45.8 ms, sys: 4.54 ms, total: 50.4 ms
Wall time: 18 ms


Fortunately, there is indeed some improvement with regard to $R^{2}_{oos}$ as expected.

### GLM

In [50]:
from group_lasso import GroupLasso

def flatten(l):
    return [item for sublist in l for item in sublist]

def SplineTransform(data,knots=3):
    spline_data = pd.DataFrame(np.ones((data.shape[0],1)),index=data.index,columns=['const'])
    for i in data.columns:
        i_dat = data.loc[:,i]
        i_sqr = i_dat**2
        i_cut, bins = pd.cut(i_dat, 3, right=True, ordered=True, retbins=True)
        i_dum = pd.get_dummies(i_cut)
        for j in np.arange(knots):
            i_dum.iloc[:,j] = i_dum.iloc[:,j]*((i_dat-bins[j])**2)
        i_dum.columns = [f"{i}_{k}" for k in np.arange(1,knots+1)]
        spline_data = pd.concat((spline_data,i_dat,i_dum),axis=1)
    return spline_data

class GLMRegression:
    
    def __init__(self,knots=3,lmd=1e-4,l1_reg=1e-4,random_state=12308):
        self.knots = knots
        self.lmd = lmd
        self.random_state = random_state
        self.l1_reg = l1_reg
        
    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
    
    def fit(self,X,y):
        groups = [0]+flatten([list(np.repeat(i,self.knots+1))[:] for i in np.arange(1,X.shape[1]+1)])
        X = SplineTransform(X)
        self.mod = GroupLasso(
            groups=groups,group_reg=self.lmd,l1_reg=self.l1_reg,
            fit_intercept=False,random_state=self.random_state
        )
        self.mod = self.mod.fit(X,y)
        return self
    
    def predict(self,X):
        X = SplineTransform(X)
        return self.mod.predict(X)

In [51]:
%%time

params = {
    'knots':[3],
    'lmd':[1e-4,1e-1],#list(np.linspace(1e-4,1e-1,10)),
    'l1_reg':[1e-4,0]
}
GLM = val_fun(GLMRegression,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'knots': 3, 'l1_reg': 0.0001, 'lmd': 0.0001} finished.
with out-of-sample R squared on validation set: 0.91194%
************************************************************
Model with params: {'knots': 3, 'l1_reg': 0.0001, 'lmd': 0.1} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************
Model with params: {'knots': 3, 'l1_reg': 0, 'lmd': 0.0001} finished.
with out-of-sample R squared on validation set: 0.86900%
************************************************************
Model with params: {'knots': 3, 'l1_reg': 0, 'lmd': 0.1} finished.
with out-of-sample R squared on validation set: 0.00000%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'knots': 3, 'l1_reg': 0.0001, 'lmd': 0.0001}
with R2oos 0.91% on validation set.
####################################################

In [52]:
np.sum(GLM.mod.sparsity_mask)

474

In [53]:
gc.collect()

0

In [54]:
%%time
evaluate(y_trn, GLM.predict(X_trn), insample=True) 
evaluate(y_tst, GLM.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 3.01%
The in-sample MSE is 0.011
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.73%
The out-of-sample MSE is 0.009
CPU times: user 2min 19s, sys: 21.4 s, total: 2min 40s
Wall time: 2min 41s


In [55]:
gc.collect()

0

### Tree-based Models

In this notebook, the best models are tree-based models, including LGBM, Random Forest and XGBoost. Quoted from the [first place solution](https://www.kaggle.com/competitions/ubiquant-market-prediction/discussion/338220) for [Ubiquant Market Prediction Competition](https://www.kaggle.com/competitions/ubiquant-market-prediction/overview): LGBM is a powerful model whose performance has been proven in many competitions. It was also the most stable (especially the consistency of CV and LB) and excellent in the experiment on the competition data.

### LightGBM

From this section till XGBoost, we will explore some tree-based models. We skip the single decision tree, as it is highly sensitive to the dataset. We first try gradient boosting decision tree (GBDT). The rationale behind is that several weak learners together result in a better performance than a single sophisticated learner. I use LightGBM to accelerate the training process while reserve the accuracy.

[LightGBM](https://lightgbm.readthedocs.io/en/v3.3.3/index.html) is a gradient boosting framework that uses tree based learning algorithms. It is designed to be distributed and efficient with the following advantages:

- Faster training speed and higher efficiency.

- Lower memory usage.

- Better accuracy.

- Support of parallel, distributed, and GPU learning.

- Capable of handling large-scale data.

LightGBM speeds up the training process of convenional gradient boosting decision tree by up to over 20 times while achieving almost the same accuracy.

In [56]:
# huber loss function for customized objective function

# gradient of huber loss with respect to y_pred
def grad_huber_obj(y_true, y_pred):
    xi = 1.35 
    # Though I do not want to make it hard-coded, lightgbm, behind the scene, evaluates the # of parameters
    # of the objective function first, then pass according # of parameters. I tried to use partial to set 
    # the value of xi. It did not work.
    # I refer the readers to the source code to have a better understanding of the issue:
    # (https://github.com/microsoft/LightGBM/blob/master/python-package/lightgbm/sklearn.py)
    y_true, y_pred = np.array(y_true).flatten(), np.array(y_pred).flatten()
    N = len(y_true)
    resid = y_true - y_pred
    ind_m = np.where(np.abs(resid)<=xi)
    ind_u = np.where(resid>xi)
    ind_l = np.where(resid< -xi)
    grad = np.zeros(N)
    try:
        grad[ind_m] = (-2*(y_true-y_pred))[ind_m]
    except:
        pass
    try:
        grad[ind_u] = np.repeat(2*xi,N)[ind_u]
    except:
        pass
    try:
        grad[ind_l] = np.repeat(-2*xi,N)[ind_l]
    except:
        pass
    return grad/N

# hessian of huber loss with respect to y_pred
def hess_huber_obj(y_true, y_pred):
    xi = 1.35
    y_true, y_pred = np.array(y_true).flatten(), np.array(y_pred).flatten()
    N = len(y_true)
    resid = y_true - y_pred
    ind_m = np.where(np.abs(resid)<=xi)
    ind_u = np.where(resid>xi)
    ind_l = np.where(resid< -xi)
    hess = np.zeros(N)
    try:
        hess[ind_m] = np.repeat(2,N)[ind_m]
    except:
        pass
    return hess/N

# huber loss for lgbm
def huber_obj(y_true, y_pred):
    grad = grad_huber_obj(y_true, y_pred)
    hess = hess_huber_obj(y_true, y_pred)
    return grad, hess

In [57]:
%%time
from lightgbm import LGBMRegressor

params = {
    'objective':[None, huber_obj],
    'max_depth':[1,2],
    'n_estimators':[10,50,100,200,500,1000],
    'random_state':[12308],
    'learning_rate':[.01,.1]
}
LGBM = val_fun(LGBMRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.87070%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10, 'objective': <function huber_obj at 0x7f7e181e21f0>, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.23432%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.92921%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50, 'objective': <function huber_obj at 0x7f7e181e21f0>, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.5167

Model with params: {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 200, 'objective': <function huber_obj at 0x7f7e181e21f0>, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.31480%
************************************************************
Model with params: {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 500, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.36812%
************************************************************
Model with params: {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 500, 'objective': <function huber_obj at 0x7f7e181e21f0>, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.43165%
************************************************************
Model with params: {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 1000, 'objective': None, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.576

In [58]:
gc.collect()

649

In [59]:
%%time
evaluate(y_trn, LGBM.predict(X_trn), insample=True) 
evaluate(y_tst, LGBM.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 19.41%
The in-sample MSE is 0.009
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 1.26%
The out-of-sample MSE is 0.009
CPU times: user 2.29 s, sys: 898 ms, total: 3.19 s
Wall time: 944 ms


In [60]:
gc.collect()

8

### Random Forest

In [61]:
%%time
from sklearn.ensemble import RandomForestRegressor

params = {
    'n_estimators': [300],
    'max_depth': [3, 6],
    'max_features': [30, 50, 100],
    'random_state': [12308]
}
RF = val_fun(RandomForestRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'max_depth': 3, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.00153%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 50, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.30646%
************************************************************
Model with params: {'max_depth': 3, 'max_features': 100, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 1.47756%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 30, 'n_estimators': 300, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 0.01128%
************************************************************
Model with params: {'max_depth': 6, 'max_features': 50, 'n_estimators': 300, 'random_st

In [62]:
gc.collect()

96

In [63]:
%%time
time.sleep(10)

evaluate(y_trn, RF.predict(X_trn), insample=True) 
evaluate(y_tst, RF.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 19.96%
The in-sample MSE is 0.009
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 2.29%
The out-of-sample MSE is 0.008
CPU times: user 1.86 s, sys: 276 ms, total: 2.14 s
Wall time: 11.7 s


In [64]:
gc.collect()

48

### XGBoost

Typically, the winner models of most Kaggle competitions are XGBoost, Neural Nets, or the ensemble model of these two. Therefore, I give XGBoost a try here, though it is not included in the paper. Like other additive tree-based models, XGBoost model often achieves higher accuracy than a single decision tree. However, it sacrifices the intrinsic interpretability of decision trees. Typically, it is easy to encounter overfitting issue when using XGBoost, especially in a low signal-to-noise ratio context. Therefore, here I try some simple settings.

In [65]:
%%time
from xgboost import XGBRegressor

params = {
    'n_estimators': [500,600,800,1000],
    'max_depth': [1,2],
    'random_state': [12308],
    'learning_rate': [.01]
}
XGB = val_fun(XGBRegressor,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 2.50783%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 600, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 2.43211%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 800, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 2.33577%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 1000, 'random_state': 12308} finished.
with out-of-sample R squared on validation set: 2.02993%
************************************************************
Model with params: {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 

In [66]:
gc.collect()

100

In [67]:
%%time

evaluate(y_trn, XGB.predict(X_trn), insample=True) 
evaluate(y_tst, XGB.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 14.87%
The in-sample MSE is 0.010
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.50%
The out-of-sample MSE is 0.008
CPU times: user 9.41 s, sys: 35 ms, total: 9.45 s
Wall time: 1.21 s


In [68]:
gc.collect()

0

### Neural Networks

Next, we consider Neural Networks. As neural nets are highly parameterised, it is easy to overfit. I use the regularization methods mentioned in the paper, say _learning rate shrinkage_ (incorporated in Adam solver), _early stopping_, _batch normaliztion_ and _ensembles_. Besides, another requirement from so many parameters is more data. Therefore, if you use tiny dataset to have a taste of neural nets, it is very likely that it underperform simpler models. However, unlike the paper, my NN5 has best out-of-sample performance.

In [69]:
import random
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, BatchNormalization
from keras.regularizers import L1L2
from keras.optimizers import Adam

# customized metrics
# out-of-sample r squared for keras
def R_oos_tf(y_true, y_pred):
    resid = tf.square(y_true-y_pred)
    denom = tf.square(y_true)
    return 1 - tf.divide(tf.reduce_mean(resid),tf.reduce_mean(denom))

# data standardization
# please standardize the data if BatchNormalization is not used
def standardize(X_trn, X_vld, X_tst):
    mu_trn = np.mean(np.array(X_trn),axis=0).reshape((1,X_trn.shape[1]))
    sigma_trn = np.std(np.array(X_trn),axis=0).reshape((1,X_trn.shape[1]))

    X_trn_std = (np.array(X_trn)-mu_trn)/sigma_trn
    X_vld_std = (np.array(X_vld)-mu_trn)/sigma_trn
    X_tst_std = (np.array(X_tst)-mu_trn)/sigma_trn
    return X_trn_std, X_vld_std, X_tst_std

# NN class
class NN:
    
    def __init__(
        self, n_layers=1, loss='mse', l1=1e-5, l2=0, learning_rate=.01, BatchNormalization=True, patience=5,
        epochs=100, batch_size=3000, verbose=1, random_state=12308, monitor='val_R_oos_tf', base_neurons=5
    ):
        self.n_layers = n_layers
        self.l1 = l1
        self.l2 = l2
        self.learning_rate = learning_rate
        self.BatchNormalization = BatchNormalization
        self.patience = patience
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.random_state = random_state
        self.monitor = monitor
        self.base_neurons = base_neurons

    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self
    
    def fit(self, X_trn, y_trn, X_vld, y_vld):
        # fix random seed for reproducibility
        random.seed(self.random_state)
        np.random.seed(self.random_state)
        tf.random.set_seed(self.random_state)
        
        # model construction
        mod = Sequential()
        mod.add(Input(shape=(X_trn.shape[1],)))
        
        for i in np.arange(self.n_layers,0,-1):
            if self.n_layers>self.base_neurons:
                if self.n_layers == i:
                    mod.add(Dense(2**i, activation='relu'))
                else:
                    mod.add(Dense(2**i, activation='relu', kernel_regularizer=L1L2(self.l1,self.l2)))
            else:
                if self.n_layers == i:
                    mod.add(Dense(2**(self.base_neurons-(self.n_layers-i)), activation='relu'))
                else:
                    mod.add(Dense(2**(self.base_neurons-(self.n_layers-i)), 
                                  activation='relu', kernel_regularizer=L1L2(self.l1,self.l2)))
            if self.BatchNormalization:
                mod.add(BatchNormalization())
        
        mod.add(Dense(1, kernel_regularizer=L1L2(self.l1,self.l2)))
        
        # early stopping
        earlystop = tf.keras.callbacks.EarlyStopping(monitor=self.monitor, patience=self.patience)

        # Adam solver
        opt = Adam(learning_rate=self.learning_rate)
        
        # compile the model
        mod.compile(loss=self.loss,
                    optimizer=opt,
                    metrics=[R_oos_tf])

        # fit the model
        mod.fit(X_trn, np.array(y_trn).reshape((len(y_trn),1)), epochs=self.epochs, batch_size=self.batch_size, 
                callbacks=[earlystop], verbose=self.verbose, 
                validation_data=(X_vld,np.array(y_vld).reshape((len(y_vld),1))))
        
        self.model = mod
        return self
    
    def predict(self, X):
        return self.model.predict(X, verbose=self.verbose)

2023-01-06 17:23:29.911495: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [70]:
%%time
# NN1-Regression-[32(relu)-1(linear)]

params = {
    'n_layers': [1],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN1 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

2023-01-06 17:23:32.181889: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -1.03620%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -1120.98877%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 1, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -74

In [71]:
gc.collect()

15924

In [72]:
%%time

evaluate(y_trn, NN1.predict(X_trn), insample=True) 
evaluate(y_tst, NN1.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is -34.16%
The in-sample MSE is 0.015
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 3.53%
The out-of-sample MSE is 0.010
CPU times: user 4.95 s, sys: 1.04 s, total: 5.98 s
Wall time: 4.21 s


In [73]:
gc.collect()

1286

In [74]:
%%time
# NN2-Regression-[32(relu)-16(relu)-1(linear)]
params = {
    'n_layers': [2],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN2 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 2, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -312.10330%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 2, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -177.64599%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 2, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -8

In [75]:
gc.collect()

3077

In [76]:
%%time

evaluate(y_trn, NN2.predict(X_trn), insample=True) 
evaluate(y_tst, NN2.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 6.39%
The in-sample MSE is 0.011
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 2.14%
The out-of-sample MSE is 0.008
CPU times: user 5.15 s, sys: 1.01 s, total: 6.16 s
Wall time: 4.16 s


In [77]:
gc.collect()

1286

In [78]:
%%time
# NN3-Regression-[32(relu)-16(relu)-8(relu)-1(linear)]
params = {
    'n_layers': [3],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN3 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 3, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -301.55560%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 3, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -273.78096%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 3, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 0.

In [79]:
gc.collect()

6804

In [80]:
%%time

evaluate(y_trn, NN3.predict(X_trn), insample=True) 
evaluate(y_tst, NN3.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is -7.04%
The in-sample MSE is 0.012
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -8.72%
The out-of-sample MSE is 0.009
CPU times: user 5.08 s, sys: 793 ms, total: 5.87 s
Wall time: 4.03 s


In [81]:
gc.collect()

1286

In [82]:
%%time
# NN4-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-1(linear)]
params = {
    'n_layers': [4],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN4 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 4, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -35.57422%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 4, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -25.37831%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 4, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 1.02

In [83]:
gc.collect()

14362

In [84]:
%%time

evaluate(y_trn, NN4.predict(X_trn), insample=True) 
evaluate(y_tst, NN4.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is -12.20%
The in-sample MSE is 0.013
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -1.61%
The out-of-sample MSE is 0.009
CPU times: user 5.26 s, sys: 795 ms, total: 6.05 s
Wall time: 4.21 s


In [85]:
gc.collect()

1286

In [86]:
%%time
# NN5-Regression-[32(relu)-16(relu)-8(relu)-4(relu)-2(relu)-1(linear)]
params = {
    'n_layers': [5],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN5 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 5, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -16.20821%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 5, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 0.89422%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 5, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 1.4786

In [87]:
gc.collect()

16121

In [88]:
%%time

evaluate(y_trn, NN5.predict(X_trn), insample=True) 
evaluate(y_tst, NN5.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is -2.50%
The in-sample MSE is 0.012
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -0.03%
The out-of-sample MSE is 0.009
CPU times: user 5.31 s, sys: 780 ms, total: 6.09 s
Wall time: 4.28 s


In [89]:
gc.collect()

1286

In [90]:
%%time
# Wide-NN1-Regression-[1024(relu)-1(linear)]

import random
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, BatchNormalization
from keras.regularizers import L1L2
from keras.optimizers import Adam

c = 10
l1 = 1e-2
l2 = 1e-2
monitor = 'val_R_oos_tf'
patience = 5
learning_rate = 1e-1
epochs = 100
batch_size = int(X_trn.shape[0]/50)
verbose = 1
random_state = 12308

# fix random seed for reproducibility
random.seed(random_state)
np.random.seed(random_state)
tf.random.set_seed(random_state)

NN1W = Sequential()
NN1W.add(Input(X_trn.shape[1]))
NN1W.add(Dense(2**c, activation='relu'))
NN1W.add(BatchNormalization())
NN1W.add(Dense(1, kernel_regularizer=L1L2(l1,l2)))

earlystop = tf.keras.callbacks.EarlyStopping(monitor=monitor, patience=patience)

opt = Adam(learning_rate=learning_rate)

# compile the model
NN1W.compile(loss='mse',
             optimizer=opt,
             metrics=[R_oos_tf])

# fit the model
NN1W.fit(X_trn, np.array(y_trn).reshape((len(y_trn),1)), epochs=epochs, batch_size=batch_size,
         callbacks=[earlystop], 
         verbose=verbose,
         validation_data=(X_vld,np.array(y_vld).reshape((len(y_vld),1))))

Epoch 1/100
50/50 [==============================] - 4s 70ms/step - loss: 43.9144 - R_oos_tf: -3655.5723 - val_loss: 5.3596 - val_R_oos_tf: -451.6760
Epoch 2/100
50/50 [==============================] - 3s 63ms/step - loss: 1.7655 - R_oos_tf: -7.6937 - val_loss: 1.5363 - val_R_oos_tf: -14.2738
Epoch 3/100
50/50 [==============================] - 3s 63ms/step - loss: 1.3056 - R_oos_tf: -0.5168 - val_loss: 1.1948 - val_R_oos_tf: -2.1730
Epoch 4/100
50/50 [==============================] - 3s 63ms/step - loss: 1.0654 - R_oos_tf: -0.2869 - val_loss: 0.9585 - val_R_oos_tf: -1.9641
Epoch 5/100
50/50 [==============================] - 3s 63ms/step - loss: 0.8354 - R_oos_tf: -0.3059 - val_loss: 0.8743 - val_R_oos_tf: -20.0934
Epoch 6/100
50/50 [==============================] - 3s 64ms/step - loss: 0.6238 - R_oos_tf: -0.2343 - val_loss: 0.7272 - val_R_oos_tf: -26.9446
CPU times: user 1min 51s, sys: 1.36 s, total: 1min 52s
Wall time: 19.8 s


In [91]:
gc.collect()

1431

In [92]:
%%time

evaluate(y_trn, NN1W.predict(X_trn,verbose=0), insample=True) 
evaluate(y_tst, NN1W.predict(X_tst,verbose=0))

***************In-Sample Metrics***************
The in-sample R2 is -1742.27%
The in-sample MSE is 0.212
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -55.03%
The out-of-sample MSE is 0.013
CPU times: user 15.7 s, sys: 612 ms, total: 16.3 s
Wall time: 11 s


In [93]:
gc.collect()

1284

In [94]:
%%time
# NN6-Regression-[64(relu)-32(relu)-16(relu)-8(relu)-4(relu)-2(relu)-1(linear)]
params = {
    'n_layers': [6],
    'loss': ['mse'],
    'l1': [1e-5,1e-3],
    'learning_rate': [.001,.01],
    'batch_size': [int(X_trn.shape[0]/50)],
    'epochs': [100],
    'random_state': [12308],
    'BatchNormalization': [True],
    'patience':[5],
    'verbose': [0],
    'monitor':['val_loss','val_R_oos_tf']
}
NN6 = val_fun(NN,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld,is_NN=True,sleep=5)

Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 6, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -129.40485%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.001, 'loss': 'mse', 'monitor': 'val_R_oos_tf', 'n_layers': 6, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: -37.60405%
************************************************************
Model with params: {'BatchNormalization': True, 'batch_size': 1920, 'epochs': 100, 'l1': 1e-05, 'learning_rate': 0.01, 'loss': 'mse', 'monitor': 'val_loss', 'n_layers': 6, 'patience': 5, 'random_state': 12308, 'verbose': 0} finished.
with out-of-sample R squared on validation set: 0.1

In [95]:
gc.collect()

22386

In [96]:
%%time

evaluate(y_trn, NN6.predict(X_trn), insample=True) 
evaluate(y_tst, NN6.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 10.72%
The in-sample MSE is 0.010
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 1.74%
The out-of-sample MSE is 0.010
CPU times: user 5.86 s, sys: 792 ms, total: 6.65 s
Wall time: 4.54 s


In [97]:
gc.collect()

1286